# Ex1) P2P Task

In [6]:
from time import sleep
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from robotshyu import Robot as rob
from tasho.OCPGenerator import OCPGenerator
import numpy as np
import casadi as cs
import tasho
import robotshyu
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env
from tasho import WorldSimulator
import pybullet as p

In [2]:
robot = rob.Robot("mmo_500_ppr")
link_name = 9
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[1, 0, 0, 1], [0, 0, 0, 0.0], [0, 0, 1, 0.7], [0, 0, 0, 1]]
    ))
q0 = [-1.0193752249977548, -0.05311582280659044, 0.0,
    1.3191046402052224, 0.0, 0.0, 0.0, 0.0, 0.0]
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.ndof
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 9-DoF robot: mmo_500_ppr


In [3]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [4]:
task_P2P.write_task_graph("MMO500_P2P_Task.svg")

In [5]:
reg_jacc = task_P2P.constraint_expressions['reg_qdd_'+robot.name].change_weight(0.1)
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1))

In [6]:
task_P2P.write_task_graph("MMO500_P2P_Task2.svg")

In [7]:
horizon_steps = 10
horizon_period = 3.0
q_pred = [0]*horizon_steps

OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      675
Number of nonzeros in inequality constraint Jacobian.:      277
Number of nonzeros in Lagrangian Hessian.............:      208

Total number of variables............................:      288
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      210
Total number of inequality constrai

In [8]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

obj = WorldSimulator.WorldSimulator()

    # Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot.name)
mmo500_frame_urdf = robotshyu.__path__[0]+"/robots/MMO_500/mmo500_description/mmo_500_ppr_frame.urdf"

frameID0 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID1 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID2 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID3 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID4 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID5 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID6 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID7 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID8 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID9 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID10 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)

robotIDs=[robotID,frameID0]
frameIDs=[frameID1,frameID2,frameID3,frameID4,frameID5,frameID6,frameID7,frameID8,frameID9,frameID10]

for i in range(horizon_steps):
    q_pred[i]=qsol[i]

joint_indices = [0, 1, 2, 13, 14, 15, 16, 17, 18]

obj.resetMultiJointState(robotIDs, joint_indices, [q0])
obj.resetMultiJointState(frameIDs, joint_indices, [q0])

obj.resetMultiJointState(frameIDs,joint_indices,q_pred)

True

In [9]:
for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetMultiJointState(
        robotIDs, joint_indices, [qsol[i]]
    )

sleep(0.5)
obj.end_simulation()

Ending simulation


# Ex2) Object picking MPC

In [7]:
robot = rob.Robot("mmo_500_ppr")
link_name = 9

# Define initial conditions of the robot
q0_val = [-1.0, -0.523598, 1.5709, 3.14, 0, -0.523598, 0.0, 0.0, 0.0]
qd0_val = [0] * robot.ndof

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)

task_P2P = P2P(robot, link_name, goal_pose, q_init, 0.001)
task_P2P.write_task_graph("MMO500_P2P_Task4.svg")

Loading robot params from json ...
Loaded 9-DoF robot: mmo_500_ppr


In [8]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_mmo_500_ppr'])
task_P2P.remove_terminal_constraints('rot_con_pose_9_mmo_500_ppr_vs_goal',
                                    'trans_con_pose_9_mmo_500_ppr_vs_goal')
task_P2P.write_task_graph("MMO500_P2P_Task5.svg")

In [9]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_mmo_500_ppr']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_9_mmo_500_ppr_vs_goal'], 1.0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_9_mmo_500_ppr_vs_goal'], 1), 
                            Regularization(task_P2P.variables['qd_mmo_500_ppr'], 1e-4))

task_P2P.write_task_graph("MMO500_P2P_Task6.svg")

In [10]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 5,"tol": 1e-3}})
# tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 0,"tol": 1e-3, "linear_solver":"ma27"}}) #use this if you have hsl


################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_mmo_500_ppr'].x
qd = pOCP.stage_tasks[0].variables['qd_mmo_500_ppr'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_mmo_500_ppr'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_mmo_500_ppr'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_mmo_500_ppr'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([0, 1, 0, 0.5]),
    cs.hcat([1, 0, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_mmo_500_ppr", "var":"qd_mmo_500_ppr", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_9_mmo_500_ppr_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

mpc_options = default_mpc_options.get_default_mpc_options()


tc.ocp_solver = "ipopt"
tc.ocp_options = mpc_options["ipopt_hsl"]["options"]
tc.mpc_solver = tc.ocp_solver
tc.mpc_options = tc.ocp_options
tc.set_ocp_solver("ipopt", tc.mpc_options)
import os; os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 218.00us ( 18.17us) 220.74us ( 18.40us)        12
       nlp_g  | 135.00us ( 11.25us) 135.82us ( 11.32us)        12
    nlp_grad  |  77.00us ( 77.00us)  76.52us ( 76.52us)         1
  nlp_grad_f  | 524.00us ( 43.67us) 527.13us ( 43.93us)        12
  nlp_hess_l  |   2.91ms (264.27us)   2.90ms (263.92us)        11
   nlp_jac_g  | 173.00us ( 14.42us) 174.15us ( 14.51us)        12
       total  |   8.64ms (  8.64ms)   8.63ms (  8.63ms)         1


sh: 1: export: : bad variable name


In [11]:
vars_db = tc.generate_MPC_component("./", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti0_p_2[12:15]-fk_T(opti0_x_1){9}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti0_p_2[12:15]-fk_T(opti0_x_1){9}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [12]:
MPC_component = MPC("mmo_500_ppr_obj_pickup", "./" + tc.name + ".json")

In [13]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot.name)
mmo500_frame_urdf = robotshyu.__path__[0]+"/robots/MMO_500/mmo500_description/mmo_500_ppr_frame.urdf"

frameID0 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID1 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID2 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID3 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID4 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID5 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID6 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID7 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID8 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)
frameID9 = obj.add_robot(position,orientation,robot_urdf=mmo500_frame_urdf)

robotIDs=[frameID0, frameID1,frameID2,frameID3,frameID4,frameID5,frameID6,frameID7,frameID8,frameID9]

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

#p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 13, 14, 15, 16, 17, 18]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")
obj.resetJointState(robotID, joint_indices, q0_val)
obj.setController(
    robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

q_log = []
q_dot_log = []
predicted_pos_log = []
q_pred = [0]*horizon_size

In [15]:
# Execute the MPC loop

p.resetBaseVelocity(cubeID, linearVelocity=[0, 1, 0])

for i in range(horizon_size * 100):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(robotID, joint_indices)
    qd_now = obj.readJointVelocities(robotID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_mmo_500_ppr"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_mmo_500_ppr"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([0, 0, 1, predicted_pos[0]]),
        cs.hcat([0, 1, 0, predicted_pos[1]]),
        cs.hcat([-1, 0, 0, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_mmo_500_ppr"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()
    sol = MPC_component.res_vals
    for i in range(horizon_size):
        q_pred[i]=sol[robot.ndof*i:robot.ndof*i+robot.ndof].full()
    obj.resetMultiJointState(robotIDs[1:], joint_indices, q_pred)

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_mmo_500_ppr"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_mmo_500_ppr"]["val"] * t_mpc).full()
    obj.setController(
        robotID, "velocity", joint_indices, targetVelocities=qd_control_sig + qdd_control_sig
    )

    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
#     if "termination_criteria_true" in MPC_component.event_output_port:
#         break

----------- MPC execution -----------
Predicted position of cube [0.49988550218801, 1.4743068474223, 0.084989896554887]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 145.00us ( 24.17us) 145.59us ( 24.27us)         6
       nlp_g  |  93.00us ( 15.50us)  91.54us ( 15.26us)         6
    nlp_grad  |  98.00us ( 98.00us)  98.32us ( 98.32us)         1
  nlp_grad_f  | 339.00us ( 56.50us) 340.21us ( 56.70us)         6
  nlp_hess_l  |   1.71ms (341.20us)   1.70ms (340.64us)         5
   nlp_jac_g  | 118.00us ( 19.67us) 117.01us ( 19.50us)         6
       total  |  21.40ms ( 21.40ms)   5.39ms (  5.39ms)         1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   2.61ms (652.00us)  85.10us ( 21.28us)         4
       nlp_g  |  46.00us ( 11.50us)  43.52us ( 10.88us)         4
    nlp_grad  |  66.00us ( 66.00us)  66.53us ( 66.53us)         1
  nlp_grad_f  | 416.00us (104.00us) 194.89us ( 48.72us)         4
  nlp_hess_l  |   7.74

----------- MPC execution -----------
Predicted position of cube [0.4998855633578, 1.6652020606885, 0.084989916510894]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.80ms (449.00us)  89.03us ( 22.26us)         4
       nlp_g  |  58.00us ( 14.50us)  58.16us ( 14.54us)         4
    nlp_grad  |  83.00us ( 83.00us)  82.41us ( 82.41us)         1
  nlp_grad_f  |   1.38ms (345.00us) 226.17us ( 56.54us)         4
  nlp_hess_l  |  12.99ms (  4.33ms)   1.05ms (350.71us)         3
   nlp_jac_g  |  63.00us ( 15.75us)  63.71us ( 15.93us)         4
       total  |  22.73ms ( 22.73ms)   3.83ms (  3.83ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49988557074615, 1.6783987447665, 0.084989917577049]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  92.00us ( 23.00us)  93.43us ( 23.36us)         4
       nlp_g  |  68.00us ( 17.00us)  65.90us ( 16.48us)         4
    nlp_grad  |   2.25ms (  2.25ms)

----------- MPC execution -----------
Predicted position of cube [0.499885707742, 1.8182344218817, 0.0849900009747]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  88.00us ( 22.00us)  89.24us ( 22.31us)         4
       nlp_g  |  56.00us ( 14.00us)  55.85us ( 13.96us)         4
    nlp_grad  |  95.00us ( 95.00us)  95.42us ( 95.42us)         1
  nlp_grad_f  | 204.00us ( 51.00us) 204.06us ( 51.02us)         4
  nlp_hess_l  |   1.00ms (334.33us)   1.00ms (335.00us)         3
   nlp_jac_g  |  78.00us ( 19.50us)  78.95us ( 19.74us)         4
       total  |   8.43ms (  8.43ms)   3.65ms (  3.65ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49988571243065, 1.8234515246465, 0.084990000306338]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  89.00us ( 22.25us)  90.22us ( 22.56us)         4
       nlp_g  |  65.00us ( 16.25us)  64.63us ( 16.16us)         4
    nlp_grad  | 660.00us (660.00us)  9

----------- MPC execution -----------
Predicted position of cube [0.49988767649201, 1.845932256609, 0.084989917225987]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  90.00us ( 22.50us)  90.32us ( 22.58us)         4
       nlp_g  |  46.00us ( 11.50us)  46.40us ( 11.60us)         4
    nlp_grad  |  91.00us ( 91.00us)  92.05us ( 92.05us)         1
  nlp_grad_f  | 673.00us (168.25us) 213.55us ( 53.39us)         4
  nlp_hess_l  |   2.88ms (959.33us)   1.01ms (336.65us)         3
   nlp_jac_g  | 385.00us ( 96.25us)  78.29us ( 19.57us)         4
       total  |  20.59ms ( 20.59ms)   3.69ms (  3.69ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49988767678858, 1.8459322564483, 0.084989917174227]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   8.10ms (  2.02ms)  99.84us ( 24.96us)         4
       nlp_g  |  54.00us ( 13.50us)  55.12us ( 13.78us)         4
    nlp_grad  |  91.00us ( 91.00us)

----------- MPC execution -----------
Predicted position of cube [0.49988767590412, 1.8459322528521, 0.084989913407198]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  92.00us ( 23.00us)  91.18us ( 22.80us)         4
       nlp_g  |  55.00us ( 13.75us)  55.40us ( 13.85us)         4
    nlp_grad  |   1.75ms (  1.75ms) 103.85us (103.85us)         1
  nlp_grad_f  | 215.00us ( 53.75us) 213.85us ( 53.46us)         4
  nlp_hess_l  |   1.00ms (334.67us)   1.01ms (335.59us)         3
   nlp_jac_g  |  78.00us ( 19.50us)  79.20us ( 19.80us)         4
       total  |  13.34ms ( 13.34ms)   3.68ms (  3.68ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49988767590412, 1.8459322528521, 0.084989913407184]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 724.00us (181.00us)  87.50us ( 21.87us)         4
       nlp_g  |  56.00us ( 14.00us)  54.57us ( 13.64us)         4
    nlp_grad  |  92.00us ( 92.00us

...

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 406.00us ( 27.07us) 304.03us ( 20.27us)        15
       nlp_g  | 391.00us ( 26.07us) 225.59us ( 15.04us)        15
    nlp_grad  |  73.00us ( 73.00us)  72.78us ( 72.78us)         1
  nlp_grad_f  |   1.02ms ( 68.20us) 705.70us ( 47.05us)        15
  nlp_hess_l  |  16.13ms (  1.15ms)   4.29ms (306.23us)        14
   nlp_jac_g  | 475.00us ( 31.67us) 269.55us ( 17.97us)        15
       total  |  41.65ms ( 41.65ms)  12.09ms ( 12.09ms)         1
----------- MPC execution -----------
Predicted position of cube [-36.635309134367, 34.721484474135, 0.084989293878384]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 952.00us ( 56.00us) 366.39us ( 21.55us)        17
       nlp_g  | 205.00us ( 12.06us) 206.41us ( 12.14us)        17
    nlp_grad  |  65.00us ( 65.00us)  64.50us ( 64.50us)         1
  nlp_grad_f  |   4.68ms (275.12us) 870.71us ( 51.22us)        17
  nlp_hess_l  |  25.79

----------- MPC execution -----------
Predicted position of cube [-34.068975130325, 32.322701694076, 0.084985256488611]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 363.00us ( 25.93us) 366.84us ( 26.20us)        14
       nlp_g  | 187.00us ( 13.36us) 185.04us ( 13.22us)        14
    nlp_grad  |  85.00us ( 85.00us)  85.32us ( 85.32us)         1
  nlp_grad_f  | 860.00us ( 61.43us) 865.52us ( 61.82us)        14
  nlp_hess_l  |  13.10ms (  1.01ms)   5.09ms (391.49us)        13
   nlp_jac_g  | 264.00us ( 18.86us) 267.78us ( 19.13us)        14
       total  |  48.12ms ( 48.12ms)  15.69ms ( 15.69ms)         1
----------- MPC execution -----------
Predicted position of cube [-33.951795267769, 32.205748803278, 0.084984955211193]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 270.00us ( 24.55us) 271.77us ( 24.71us)        11
       nlp_g  | 132.00us ( 12.00us) 133.30us ( 12.12us)        11
    nlp_grad  |  98.00us ( 98.00us

----------- MPC execution -----------
Predicted position of cube [-32.719838738362, 31.072286678434, 0.084989916361583]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 281.00us ( 25.55us) 245.35us ( 22.30us)        11
       nlp_g  | 171.00us ( 15.55us) 127.44us ( 11.59us)        11
    nlp_grad  |  93.00us ( 93.00us)  93.79us ( 93.79us)         1
  nlp_grad_f  |   9.29ms (844.45us) 602.54us ( 54.78us)        11
  nlp_hess_l  |  52.35ms (  5.24ms)   3.40ms (340.32us)        10
   nlp_jac_g  | 190.00us ( 17.27us) 189.67us ( 17.24us)        11
       total  |  72.25ms ( 72.25ms)  10.13ms ( 10.13ms)         1
----------- MPC execution -----------
Predicted position of cube [-32.673282853151, 31.023518737339, 0.084989483603709]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 280.00us ( 23.33us) 280.18us ( 23.35us)        12
       nlp_g  | 139.00us ( 11.58us) 140.42us ( 11.70us)        12
    nlp_grad  |  74.00us ( 74.00us

----------- MPC execution -----------
Predicted position of cube [-32.175186124128, 30.596222059961, 0.084989968359983]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 217.00us ( 21.70us) 219.91us ( 21.99us)        10
       nlp_g  | 128.00us ( 12.80us) 125.98us ( 12.60us)        10
    nlp_grad  |  97.00us ( 97.00us)  97.11us ( 97.11us)         1
  nlp_grad_f  | 515.00us ( 51.50us) 516.76us ( 51.68us)        10
  nlp_hess_l  |   3.00ms (333.67us)   3.00ms (333.74us)         9
   nlp_jac_g  | 192.00us ( 19.20us) 193.05us ( 19.31us)        10
       total  |  24.24ms ( 24.24ms)   8.67ms (  8.67ms)         1


error: Not connected to physics server.

In [16]:
obj.end_simulation()

max(MPC_component.solver_times)

Ending simulation


error: Not connected to physics server.

# Ex3) Nonholonomic Constraint

In [ ]:
# Define initial conditions of the robot
q0_val = [-1.0, -0.523598, 3.14, 3.14, 0, -0.523598, 0.0, 0.0, 0.0]
qd0_val = [0] * robot.ndof

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, link_name, goal_pose, q_init, 0.001)
task_P2P.write_task_graph("MMO500_P2P_Task4.svg")

In [ ]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_mmo_500_ppr'])
task_P2P.remove_terminal_constraints('rot_con_pose_9_mmo_500_ppr_vs_goal',
                                    'trans_con_pose_9_mmo_500_ppr_vs_goal')
task_P2P.write_task_graph("MMO500_P2P_Task5.svg")

In [ ]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_mmo_500_ppr']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_9_mmo_500_ppr_vs_goal'], 1.0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_9_mmo_500_ppr_vs_goal'], 1), 
                            Regularization(task_P2P.variables['qd_mmo_500_ppr'], 1e-4))

task_P2P.write_task_graph("MMO500_P2P_Task6.svg")

In [ ]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 5,"tol": 1e-3}})
# tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 0,"tol": 1e-3, "linear_solver":"ma27"}}) #use this if you have hsl


################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_mmo_500_ppr'].x
qd = pOCP.stage_tasks[0].variables['qd_mmo_500_ppr'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_mmo_500_ppr'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_mmo_500_ppr'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_mmo_500_ppr'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([0, 1, 0, 0.5]),
    cs.hcat([1, 0, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_mmo_500_ppr", "var":"qd_mmo_500_ppr", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_9_mmo_500_ppr_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

mpc_options = default_mpc_options.get_default_mpc_options()


tc.ocp_solver = "ipopt"
tc.ocp_options = mpc_options["ipopt_hsl"]["options"]
tc.mpc_solver = tc.ocp_solver
tc.mpc_options = tc.ocp_options
tc.set_ocp_solver("ipopt", tc.mpc_options)
import os; os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

In [ ]:
vars_db = tc.generate_MPC_component("./", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

In [ ]:
MPC_component = MPC("mmo_500_ppr_obj_pickup", "./" + tc.name + ".json")

In [ ]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
mmoID = obj.add_robot(position, orientation, "mmo_500_ppr")

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

#p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 13, 14, 15, 16, 17, 18]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")
obj.resetJointState(robotID, joint_indices, q0_val)
obj.setController(
    robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

In [ ]:
# Execute the MPC loop
q_log = []
q_dot_log = []
predicted_pos_log = []

p.resetBaseVelocity(cubeID, linearVelocity=[0, 1, 0])

for i in range(horizon_size * 100):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(robotID, joint_indices)
    qd_now = obj.readJointVelocities(robotID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_mmo_500_ppr"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_mmo_500_ppr"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([0, 0, 1, predicted_pos[0]]),
        cs.hcat([0, 1, 0, predicted_pos[1]]),
        cs.hcat([-1, 0, 0, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_mmo_500_ppr"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_mmo_500_ppr"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_mmo_500_ppr"]["val"] * t_mpc).full()
    obj.setController(
        robotID, "velocity", joint_indices, targetVelocities=qd_control_sig + qdd_control_sig
    )

    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
    if "termination_criteria_true" in MPC_component.event_output_port:
        break

In [ ]:
obj.end_simulation()

max(MPC_component.solver_times)